In [9]:
import gymnasium as gym
import numpy as np
import gymnasium as gym

In [10]:
### NeuralNet
def derivative_bcel(prediction, ground_truth):
    x =  np.where(ground_truth == 0, 1.0 / (1.0 - prediction), -1.0 / prediction)
    return x

def derivative_mse(prediction, ground_truth):
    batch_size = ground_truth.shape[0]
    return -2 * (ground_truth - prediction) / batch_size

class Linear:
    def __init__(self, input_size, output_size):
        self.input_size = input_size
        self.output_size = output_size
        self.W = np.random.uniform(-1, 1,(self.input_size,self.output_size))
        self.B = np.zeros((1, self.output_size))
    
    def forward(self, x):
        self.fw = x
        return np.dot(x, self.W) + self.B
    
    def backward(self, d, lr):
        d_w = np.dot(self.fw.T, d)
        d_e = np.dot(d, self.W.T)
        d_b = np.sum(d, axis=0, keepdims=True)
        self.W -= lr * d_w / self.fw.shape[0]
        self.B -= lr * d_b / self.fw.shape[0]
        return d_e


class ReLU:
    def __init__(self):
        pass

    def forward(self, x):
        self.fw = x
        return x * (x > 0)
    
    def backward(self, d, lr):
        return d * np.where(self.fw > 0, 1.0, 0.0)
    

class Sigmoid:
    def __init__(self):
        pass
    
    def forward(self, x):
        self.fw = x
        self.out = 1.0 / (1.0 + np.exp(-x))
        return self.out
    
    def backward(self, d, lr):
        return d * (self.out * (1.0 - self.out))
    

class NeuralNetwork:
    def __init__(self, layers: list):
        self.layers = layers

    def forward_pass(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x
    
    def backward_pass(self, deriv, lr):
        for layer in reversed(self.layers):
            deriv = layer.backward(deriv, lr)



In [11]:
env = gym.make('CartPole-v1')
num_actions = env.action_space.n
obs_shape = env.observation_space.shape[0]

In [13]:
class Individual:
    def __init__(self, n_obs, n_act):
        q_net = NeuralNetwork([
            Linear(1, 32), ReLU(),
            Linear(32, 32), ReLU(),
            Linear(32, 1) ])
        self.fitness = -np.inf

    def mutate(self, rate):
        if np.random.random() < rate:
            for layer in self.net.layers:
                if type(layer) == Linear:
                    layer.W += np.random.normal(0,1, size= layer.W.shape)
                    layer.B += np.random.normal(0,1, size = layer.B.shape)
    def fitness(self, func: callable):
        self.fitness = func()
